In [1]:
import pandas as pd
import xarray as xr
import numpy as np
from linopy import Model

# Technologies data:
tech_param = pd.read_csv('Data/technologies.csv',index_col=0)
# Storages data
sto_param = pd.read_csv("Data/storage_technologies.csv", index_col=0).to_xarray()
sto_param_cold = pd.read_csv("Data/storage_cold_technologies.csv", index_col=0).to_xarray()
# Demand:
demand = pd.read_csv('Data/demand_hourly_Abu_Dhabi.csv',index_col=0, delimiter = ';').to_xarray()
# COP
cop = pd.read_csv('Data/COP.csv',index_col=0, delimiter= ',')
# Global parameters:
general_parameters = pd.read_csv('Data/global_parameters.csv',index_col=0, delimiter= ',')
# Airchiller data:
conv_tech_param = pd.read_csv('Data/conversion_technologies.csv', index_col =0, delimiter = ',').to_xarray()

# Renewable generation:
AF = pd.read_csv('Data/AF.csv',index_col=0, delimiter = ';') # AF is given in the file only for the renewables, we need to add columns for the other technologies.
for tech in tech_param.index:
    if tech not in AF.columns:
        AF.loc[:,tech] = tech_param.loc[tech, "Availability"] 
tech_param = tech_param.to_xarray()

In [2]:
### Sets
Time = demand.get_index("Time")
Tech = tech_param.get_index("Technologies")
Locations = pd.Index(["Yas", "Mariah", "Muroor", "Reem"], name="Locations")  # creation of my set of each location

# Technologies related
TECH_MAX_CAP = tech_param['Maximum capacity']

MARGINAL_FUEL_COST = tech_param['Fuel costs']/tech_param['Rated efficiency']
CO2_INTENSITY = tech_param['Fuel CO2 content']/tech_param['Rated efficiency']
MARGINAL_CO2_EMISSION = tech_param['Fuel CO2 content']
TECH_PHI = tech_param["Discount rate"]/(1-(1+tech_param["Discount rate"])**(-tech_param["Lifetime"]))
TECH_IC = TECH_PHI*tech_param['Investment cost']
DELTA_MAX_UP = tech_param['Ramp-up rate']
DELTA_MAX_DOWN = tech_param['Ramp-down rate']
AF = AF[Tech]
AF  = xr.DataArray(AF,coords=[Time,Tech])
LEGACY = tech_param["Legacy capacity"]

# Electrical Storage related
STO_MAX_CAP = sto_param["Maximum capacity"]
STO_PHI = sto_param["Discount rate"]/(1-(1+sto_param["Discount rate"])**(-sto_param["Lifetime"]))
STO_IC = STO_PHI*sto_param["Investment cost"]
EFF_D = sto_param["Discharge efficiency"]
EFF_C = sto_param["Charge efficiency"]
SOC_MAX = sto_param["Maximum state of charge"]
SOC_MIN = sto_param["Minimum state of charge"]
STO_dT = sto_param["Storage duration"]

# Cold Storage related
STO_MAX_CAP_COLD = sto_param_cold["Maximum capacity"]
STO_PHI_COLD = sto_param_cold["Discount rate"]/(1-(1+sto_param_cold["Discount rate"])**(-sto_param_cold["Lifetime"]))
STO_IC_COLD = STO_PHI_COLD*sto_param_cold["Investment cost"]
EFF_D_COLD = sto_param_cold["Discharge efficiency"]
EFF_C_COLD = sto_param_cold["Charge efficiency"]
SOC_MAX_COLD = sto_param_cold["Maximum state of charge"]
SOC_MIN_COLD = sto_param_cold["Minimum state of charge"]
STO_dT_COLD = sto_param_cold["Storage duration"]


# Links Related
conv_tech_phi = conv_tech_param['Discount rate']/(1-(1+conv_tech_param['Discount rate'])**(-conv_tech_param['Lifetime']))
CONV_TECH_IC = conv_tech_phi*conv_tech_param['Investment cost']
CONV_TECH_MAX_CAP = conv_tech_param['Maximum capacity']

# demande
COP = cop["COP"]
COP_avg = COP.mean().item()                            # Use for the objective function (= 4)
COP  = xr.DataArray(COP,coords=[Time])
DEMAND_elec = demand["Campus_elec"]                    # Elec demand total in my network [MW_froid]
DEMAND_cold_total = demand["Campus_cold"]              # Cold demand total in my network [MW_froid]                 
DEMAND_cold = xr.Dataset( {"Yas": demand["Yas"],       # [MW_froid] Fixed cold demand of each VPP (= sum of the cold demande of each building in 4 locations)
                       "Mariah": demand["Mariah"],
                       "Muroor": demand["Muroor"],
                       "Reem": demand["Reem"]} ).to_array(dim="Locations")   
ELEC_DEMAND_cold_total = DEMAND_cold_total / COP             # total of the Electricity demand of cold in my network [MW_elec]

# Configuration and scenario parameters
TS = float(general_parameters.Value.loc['Time step'])
VOLL = float(general_parameters.Value.loc['VOLL'])
CO2_PRICE = float(general_parameters.Value.loc['CO2_price'])
CO2_BOUND = float(general_parameters.Value.loc['CO2_bound'])

In [3]:
### Create the model instance

m = Model()

### Create the variables

cap_tech = m.add_variables(lower = 0, coords = [Tech], name = 'cap_tech') # Installed capacity of the technologies [MW]
g = m.add_variables(lower = 0, coords = [Tech, Time], name = 'g')         # Generated power [MW]
fcd = m.add_variables(lower = 0, coords = [Tech, Time], name = 'fcd')     # Fuel cost definition [EUR/h]
ccd = m.add_variables(lower = 0, coords= [Tech, Time], name = 'ccd')      # CO2 cost definition [EUR/h]
cap_sto = m.add_variables(lower = 0, name = 'cap_sto')                    # Installed capacity of the storages [MWh]
sd = m.add_variables(lower = 0, coords = [Time], name = 'sd')             # Storage discharge [MW]
sc = m.add_variables(lower = 0, coords = [Time], name = 'sc')             # Storage charge [MW]
se = m.add_variables(lower = 0, coords = [Time], name = 'se')             # Stored energy [MWh]

co2_emissions =  m.add_variables(lower = 0, name = 'co2_emissions')       # Combined CO2 emissions [tCO2eq]
inj = m.add_variables(lower = 0, coords= [Time], name = 'inj')            # Electricity required to meet the cold demand through the conversion system [MW]
ens_elec = m.add_variables(lower = 0, coords = [Time], name = 'ens')      # Energy no served for the electric part [MWh]
ens_cold = m.add_variables(lower = 0, coords = [Time], name = 'ens_cold') # Energy no served for the cold part [MWh]
#NO curtailment considered here

#Cold battery variables
vpp_c = m.add_variables(lower = 0, coords= [Time], name = 'vpp_c')        # Storage charge (Ice formation) [MW_cold]
vpp_e = m.add_variables(lower = 0, coords= [Time], name = 'vpp_e')        # Stored energy [MWh_cold]

# optimize the size of the storage
e_max = m.add_variables(lower = 0, name = 'e_max')                        # Installed capacity of the storages [MWh_cold] 
c_max = m.add_variables(lower = 0, name = "c_max")                        # Installed capacity of the storages [MWh]



### Create the constraints

# Technology sizing
cap_max_tech = m.add_constraints(cap_tech <= TECH_MAX_CAP, name = "cap_max_tech")
power_max = m.add_constraints(g <= AF*cap_tech, name= 'power_max')

# Ramping up/down constraints
power_rampup_initial = m.add_constraints(g.loc[:,:Time[0]] - g.loc[:,Time[len(Time)-1]:] <= DELTA_MAX_UP*cap_tech , name = 'power_rampup_initial')
power_rampup = m.add_constraints(g.loc[:, Time[1:]] <= g.shift(Time = 1) + DELTA_MAX_UP*cap_tech, name = 'power_rampup')

power_rampdown_initial = m.add_constraints(g.loc[:,:Time[0]] - g.loc[:,Time[len(Time)-1]:]  >= - DELTA_MAX_DOWN*cap_tech, name = 'power_rampdown_initial')
power_rampdown = m.add_constraints(g.loc[:, Time[1:]]>= g.shift(Time = 1) - DELTA_MAX_DOWN*cap_tech, name = 'power_rampdown')


# Electrical Storage sizing + Initial condition
cap_max_sto = m.add_constraints(cap_sto <= STO_MAX_CAP, name = "cap_max_sto")
storage_balance = m.add_constraints(se.loc[Time[1:]] - se.shift(Time = 1) == TS*EFF_C*sc.loc[Time[1:]] -TS*sd.loc[Time[1:]]/EFF_D, name = 'storage_balance')
storage_charge_max = m.add_constraints(sc <= cap_sto/STO_dT, name = 'storage_charge_max')
storage_in_fin = m.add_constraints(se.loc[:Time[0]] == se.loc[Time[len(Time)-1]:] , name = 'storage_in_fin')

storage_discharge_max = m.add_constraints(sd <= cap_sto/STO_dT, name = 'storage_discharge_max')
storage_energy_max = m.add_constraints(se <= SOC_MAX*cap_sto, name = 'storage_energy_max' )
storage_energy_min = m.add_constraints(se >= SOC_MIN*cap_sto, name = 'storage_energy_min' )


#Cold storage sizing
cap_max_sto_cold = m.add_constraints(e_max <= STO_MAX_CAP_COLD, name = "cap_max_sto_cold")
storage_balance_cold = m.add_constraints(vpp_e.loc[Time[1:]] == vpp_e.loc[Time[:-1]] + TS*(EFF_C_COLD * vpp_c.loc[Time[:-1]] - DEMAND_cold_total.loc[Time[:-1]]/EFF_D_COLD), name="storage_balance_cold")
storage_circular_cold = m.add_constraints(vpp_e.loc[Time[0]] == vpp_e.loc[Time[-1]] + TS * (EFF_C_COLD * vpp_c.loc[Time[-1]] - DEMAND_cold_total.loc[Time[-1]]/EFF_D_COLD), name="storage_cyclic_cold")  

storage_charge_max_cold = m.add_constraints(vpp_c <= e_max*STO_dT_COLD, name = 'storage_charge_max_cold')   
storage_energy_max_cold = m.add_constraints(vpp_e <= SOC_MAX_COLD*e_max, name = 'storage_energy_max_cold' )
storage_energy_min_cold = m.add_constraints(vpp_e >= SOC_MIN_COLD*e_max, name = 'storage_energy_min_cold' )

# Market clearing
mcc = m.add_constraints(g.sum(dims = 'Technologies') + sd - sc == - ens_elec + DEMAND_elec + inj , name = 'mcc')
mcc2 = m.add_constraints(inj*COP + ens_cold == vpp_c, name = 'mcc2')

# Limit on energy not served
ens_lim = m.add_constraints(ens_elec<=DEMAND_elec, name = 'ens_lim')
ens_lim2 = m.add_constraints(ens_cold<=DEMAND_cold_total, name = 'ens_lim2')
links_max = m.add_constraints(inj*COP <= c_max, name="max_flow_conv_tech")

# Costs
fuel_cost = m.add_constraints(fcd == MARGINAL_FUEL_COST*g, name="fuel_cost")

# CO2 Emissions
co2_account =  m.add_constraints(co2_emissions == (CO2_INTENSITY*g).sum(dims=["Technologies", "Time"]), name="co2 account")
co2 = m.add_constraints(ccd == CO2_PRICE*MARGINAL_CO2_EMISSION*g, name = 'co2')


### Add the objective function
total_cost = (TECH_IC*cap_tech).sum(dim="Technologies") + (CONV_TECH_IC*c_max) + (STO_IC*cap_sto) + (STO_IC_COLD*e_max) + (TS*fcd + TS*ccd).sum(dims = ['Technologies', 'Time']) + (VOLL*TS*ens_elec).sum(dims ='Time') + (VOLL*TS*ens_cold).sum(dims ='Time')
m.add_objective(total_cost)

In [ ]:
m.solve("gurobi", BarHomogeneous = True, BarConvTol=1e-4, MIPGap=0.0002)

Set parameter WLSAccessID
Set parameter WLSSecret


In [ ]:
print('capacité maximum de la baterie:', cap_sto.solution.values) 
print('Valeur maximum de ma charge durant l année:', c_max.solution.values) 
print('capacité maximum du stockage de froid:', e_max.solution.values)
print('Valeur maximum de ma charge durant l année:', vpp_c.solution.max(dim='Time').values)
print('Valeur total de ma charge durant l année:', vpp_c.solution.sum().values)
print('totale de demande de froid durant l année:', DEMAND_cold_total.sum().values)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
lightblue = "#89CFF0"  
purple = "#bcbddc"
darkblue = "#003366"
red = "#c994c7"

# 0) Préparation (à exécuter dans l'environnement où vpp_c, vpp_e, DEMAND_cold_total sont disponibles)
sample_day = vpp_c.solution.Time[265:289]
hours      = np.arange(1, len(sample_day) + 1)

# 1) Séries de données
cold_demand = DEMAND_cold_total.sel(Time=sample_day)
storage = vpp_e.solution.sel(Time=sample_day)
charge = vpp_c.solution.sel(Time=sample_day)

# 2) Création de la figure 
fig, ax1 = plt.subplots(figsize=(12, 6))

ln1 = ax1.plot(hours, cold_demand, marker='o', label="Cold demand (MW)", color=purple)
ln2 = ax1.plot(hours, storage, linestyle='--', marker='s', label="Storage state (MWh)", color=lightblue)
ln3 = ax1.plot(hours, charge, linestyle='-.', marker='^', label="Cold storage charging (MW)", color=darkblue)
ax1.set_xlabel("Hour")
ax1.set_ylabel("Cold demand, charging (MW) and storage level (MWh)")
ax1.set_xticks(hours)
ax1.grid(True)
lns = ln1 + ln2 + ln3
labels = [l.get_label() for l in lns]
ax1.legend(lns, labels, loc='upper left')

plt.title("Cold demand, storage state and charging – Total campus")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Assuming 'g' is already defined and contains the solution dataframe
generation_df = g.solution.to_dataframe().unstack(level=0)
generation_df.columns = generation_df.columns.droplevel(0)

# Calculate total production for each column
total_production = generation_df.sum()

# Sort columns by total production in descending order
sorted_columns = total_production.sort_values(ascending=False).index
# Sort values within each column in descending order
generation_df = generation_df[sorted_columns].apply(lambda x: x.sort_values(ascending=False).values, axis=0)

bottom_stack = np.zeros(8760)
demand = DEMAND_elec.to_series().sort_values(ascending=False).values
#plt.fill_between(range(8760), bottom_stack, demand, label='Battery Discharge', alpha=0.7)

# Ensure the DataFrame has 8760 rows
if generation_df.shape[0] != 8760:
    raise ValueError("The DataFrame does not have 8760 rows. Check the input data.")

# Initialize the bottom stack for the stacked area plot
bottom_stack = np.zeros(8760)

# Plot the load duration curve
for column in generation_df.columns:
    plt.fill_between(range(8760), bottom_stack, bottom_stack + generation_df[column], label=column, alpha=0.7)
    bottom_stack += generation_df[column]

# Convert DEMAND DataArray to a pandas Series and sort it in descending order
demand = DEMAND_elec.to_series().sort_values(ascending=False).values
#plt.plot(range(8760), demand, color='black', label='Demand', linewidth=2)


plt.xlabel("Time [h]")
plt.ylabel("Power Generation [MW]")
plt.title("Yearly power generation")
plt.legend()
plt.grid()
plt.show()

In [ ]:
cap_tech.solution.to_dataframe().plot(kind='bar')
print(cap_tech.solution.values)